# SpringBoard Capstone2: Coronavirus Tweets Sentiment Analysis

    Data from Kaggle.com: https://www.kaggle.com/datatattle/covid-19-nlp-text-classification/metadata
## Multi_Dense_Layer used for modeling, classes reduced to 3. 
    Extremely tweets were categoried as positive or negative

In [ ]:
# ## Cell used in Kaggles webpage

# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# # import os
# # for dirname, _, filenames in os.walk('/kaggle/input'):
# #     for filename in filenames:
# #         print(os.path.join(dirname, filename))

# # # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
nltk.download('wordnet')
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
# from wordcloud import WordCloud
import re
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report,ConfusionMatrixDisplay

### references: 
### https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/
### https://github.com/mengyanl/Twitter_Sentiment_Analysis/blob/main/notebooks/1.5%20Twitter%20Sentiment%20Analysis%20-%20Bag%20of%20Words%20(Code%20Submission).ipynb

In [ ]:
tweet_whole_train = pd.read_csv('/Users/Ling/Desktop/01_Springboard/Corona_Tweet/Data/Corona_NLP_train.csv', index_col = 0)
print(tweet_whole_train.info())
test = pd.read_csv('/Users/Ling/Desktop/01_Springboard/Corona_Tweet/Data/Corona_NLP_test.csv', index_col = 0)
print(test.info())

In [ ]:
plt.figure(figsize=[15,7])
sns.histplot(data=tweet_whole_train, x = 'TweetAt', hue = 'Sentiment', multiple='dodge')
plt.xticks(rotation = 90)
plt.show()

In [ ]:
# plt.figure(figsize=[15,7])
# sns.histplot(data=test, x = 'TweetAt', hue = 'Sentiment', multiple='dodge')
# plt.xticks(rotation = 90)
# plt.show()

In [ ]:
print(len(tweet_whole_train['Location'].unique()))
print(len(test['Location'].unique()))

In [ ]:
tweet = tweet_whole_train[['OriginalTweet', 'Sentiment']]

In [ ]:
order = ['Extremely Negative', 'Negative', 'Neutral','Positive','Extremely Positive']
sns.countplot(x = 'Sentiment', data = tweet, order = order, palette=['#432371',"#FAAE7B","#7fcdbb"])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
# lower case all text
tweet['OriginalTweet'] = tweet['OriginalTweet'].str.lower()
test['OriginalTweet'] = test['OriginalTweet'].str.lower()

print('TRAIN DATA \n', tweet['OriginalTweet'].head())
print('\n TEST DATA \n', test['OriginalTweet'].head())

In [ ]:
def remove_at_signs(text):
    return " ".join([word for word in str(text).split() if not word.startswith('@')])
tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda text: remove_at_signs(text))
test['OriginalTweet'] = test['OriginalTweet'].apply(lambda text: remove_at_signs(text))

print('TRAIN DATA \n', tweet['OriginalTweet'].head())
print('\n TEST DATA \n', test['OriginalTweet'].head())

In [ ]:
import itertools
def cleaning_repeating_char(text):
    cl = ''.join(ch for ch, _ in itertools.groupby(text))
    return cl
tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda x: cleaning_repeating_char(x))
test['OriginalTweet'] = test['OriginalTweet'].apply(lambda x: cleaning_repeating_char(x))

print('TRAIN DATA \n', tweet['OriginalTweet'].head())
print('\n TEST DATA \n', test['OriginalTweet'].head())

In [ ]:
def cleaning_URLs(data):
    return re.sub('[^ ]+\.[^ ]+' , ' url', data)
tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda x: cleaning_URLs(x))
test['OriginalTweet'] = test['OriginalTweet'].apply(lambda x: cleaning_URLs(x))

print('TRAIN DATA \n', tweet['OriginalTweet'].head())
print('\n TEST DATA \n', test['OriginalTweet'].head())

In [ ]:
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans(' ', ' ', punctuations_list)
    return text.translate(translator)
tweet['OriginalTweet']= tweet['OriginalTweet'].apply(lambda x: cleaning_punctuations(x))
test['OriginalTweet']= test['OriginalTweet'].apply(lambda x: cleaning_punctuations(x))

print('TRAIN DATA \n', tweet['OriginalTweet'].head())
print('\n TEST DATA \n', test['OriginalTweet'].head())

In [ ]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda x: cleaning_numbers(x))
test['OriginalTweet'] = test['OriginalTweet'].apply(lambda x: cleaning_numbers(x))

print('TRAIN DATA \n', tweet['OriginalTweet'].head())
print('\n TEST DATA \n', test['OriginalTweet'].head())

In [ ]:
# Tokenize the OriginalTweet text
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
tweet['Tokens'] = [tknzr.tokenize(t) for t in tweet['OriginalTweet']]
tweet['Tokens'].head()

In [ ]:
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
tweet['Tokens']= tweet['Tokens'].apply(lambda x: stemming_on_text(x))
tweet['Tokens'].head(10)

In [ ]:
stop_words = ['a',
 'about',
 'above',
 'after',
 'am',
 'all',
              'al',
              
              'an',
 'and',
 'any',
 'are',
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'but',
 'by',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'down',
 'during',
 'each',
 'for',
 'from',
 'further',
 'had',
 'has',
 'have',
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 "here's",
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 "how's",
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'it',
 "it's",
 'its',
 'itself',
 "let's",
 'me',
 'my',
 'myself',
 'of',
 'off',
 'on',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'same',
 'she',
 "she'd",
 "she'll",
 "she's",
 'should',
 'so',
 'some',
 'such',
 'than',
 'that',
 "that's",
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',
 'there',
 "there's",
 'these',
 'they',
 "they'd",
 "they'll",
 "they're",
 "they've",
 'this',
 'those',
 'through',
 'to',
 'under',
 'until',
 'up',
 'very',
 'was',
 'we',
 "we'd",
 "we'll",
 "we're",
 "we've",
 'were',
 'what',
 "what's",
 'when',
 "when's",
 'where',
 "where's",
 'which',
 'while',
 'who',
 "who's",
 'whom',
 'why',
 "why's",
 'with',
 "won't",
 'would',
 'you',
 "you'd",
 "you'll",
 "you're",
 "you've",
 'your',
 'yours',
 'yourself',
 'yourselves',
 "that'll",
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ma',
 'across',
 'im',
 'l',
 'shes',
 'thatl',
 'thatll',
 'wil',
 'youd',
 'youl',
 'youll',
 'youre',
 'youve']

In [ ]:
STOPWORDS = set(stop_words)
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda text: cleaning_stopwords(text))
test['OriginalTweet'] = test['OriginalTweet'].apply(lambda text: cleaning_stopwords(text))

print('TRAIN DATA \n', tweet['OriginalTweet'].head())
print('\n TEST DATA \n', test['OriginalTweet'].head())

In [ ]:
# Tokenize the OriginalTweet text
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
tweet['Tokens'] = [tknzr.tokenize(t) for t in tweet['OriginalTweet']]
test['Tokens'] = [tknzr.tokenize(t) for t in test['OriginalTweet']]

print('TRAIN DATA \n', tweet['Tokens'].head())
print('\n TEST DATA \n', test['Tokens'].head())

In [ ]:
nltk.download('averaged_perceptron_tagger')
tag1 = nltk.pos_tag(tweet.iloc[20011,2])
tag1

In [ ]:
lm = nltk.stem.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data
tweet['Tokens'] = tweet['Tokens'].apply(lambda x: lemmatizer_on_text(x))
test['Tokens'] = test['Tokens'].apply(lambda x: lemmatizer_on_text(x))

print('TRAIN DATA \n', tweet['Tokens'].head())
print('\n TEST DATA \n', test['Tokens'].head())

In [ ]:
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
tweet['Tokens']= tweet['Tokens'].apply(lambda x: stemming_on_text(x))
tweet['Tokens'].head(10)

In [ ]:
tweet['text'] = tweet["Tokens"].map(' '.join)
test['text'] = test["Tokens"].map(' '.join)

print('TRAIN DATA \n', tweet['text'].head())
print('\n TEST DATA \n', test['text'].head())

In [ ]:
STOPWORDS = set(stop_words)
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
tweet['text'] = tweet['text'].apply(lambda text: cleaning_stopwords(text))
test['text'] = test['text'].apply(lambda text: cleaning_stopwords(text))

print('TRAIN DATA \n', tweet['text'].head())
print('\n TEST DATA \n', test['text'].head())

In [ ]:
#!pip install textblob

In [ ]:
#Adding dimensions with textblob
#TextBlob Features
from textblob import TextBlob

def tb_enrich(ls):
    #Enriches a column of text with TextBlob Sentiment Analysis outputs
    tb_polarity = []
    tb_subject = []

    for tweet in ls:
        sentiment = TextBlob(tweet).sentiment
        tb_polarity.append(sentiment[0])
        tb_subject.append(sentiment[1])
    

    return tb_polarity, tb_subject
    
tweet["Polarity"], tweet["Subjectivity"] = tb_enrich(list(tweet["text"]))
test["Polarity"], test["Subjectivity"] = tb_enrich(list(test["text"]))

In [ ]:
tweet.describe()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.model_selection import train_test_split 

X = tweet['text'] 
y = tweet['Sentiment']

Count_vect = CountVectorizer(ngram_range=(1, 1), max_df=0.1, min_df=20)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)
Count_train = Count_vect.fit_transform(X_train).toarray()
Count_test = Count_vect.transform(X_test).toarray()

# Print the features
print('# of feature names', len(Count_vect.get_feature_names()))
print(Count_vect.get_feature_names()[99::100])

In [ ]:
# #Tokenisation
# from tensorflow.keras.preprocessing.text import Tokenizer

# #Define the Tokeniser
# tokenizer = Tokenizer(num_words=3000, lower=True)

# #Create the corpus by finding the most common 
# tokenizer.fit_on_texts(tweet["text"])

# ##Train
# #Tokenise our column of edited Tweet content
# tweet_tokens = tokenizer.texts_to_matrix(list(tweet["text"]))

# #Test
# #Tokenise our column of edited Tweet content
# tweet_tokens_test = tokenizer.texts_to_matrix(list(test["text"]))


In [ ]:
#Combining the dataframe with the tokens using pd.concat

#Reset axes to avoid overlapping
# tweet.reset_index(drop=True, inplace=True)
# test.reset_index(drop=True, inplace=True)

##Train
# full_train = pd.concat([tweet[['Polarity', 'Subjectivity']], pd.DataFrame(tweet_tokens)], sort=False, axis=1)

In [ ]:
# full_train.head()

In [ ]:
# ##Train
# full_train2 = 
# full_train2.head()

# full_test2 = 
# full_train.head()

In [ ]:
#Define the indexing for each possible label in a dictionary
class_to_index = {"Neutral":1, "Positive":2, "Negative":0,"Extremely Negative":0, "Extremely Positive": 2}

#Creates a reverse dictionary
index_to_class = {1: "Neutral", 2: "Positive", 0: "Negative"}

#Creates lambda functions, applying the appropriate dictionary
names_to_ids = lambda n: np.array([class_to_index.get(x) for x in n])
ids_to_names = lambda n: np.array([index_to_class.get(x) for x in n])

#Test each function
print(names_to_ids(["Extremely Positive", "Negative", "Positive", "Neutral"]))
print(ids_to_names([0,1,2]))


#Convert the "Sentiment" column into indexes
y_train_id = names_to_ids(y_train)
y_test_id = names_to_ids(y_test)

y_test_id[0:3]

In [ ]:
y_test_id.shape

In [ ]:
#!pip install tensorflow

In [ ]:
import tensorflow as tf
train_tf = tf.convert_to_tensor(Count_train)
test_tf = tf.convert_to_tensor(Count_test)

In [ ]:
train_tf.shape

In [ ]:
# from sklearn.utils import class_weight
# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(y_train_id),
#                                                  y_train_id)

In [ ]:
#Tensorflow / Keras
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(6, input_dim=3652, activation='relu'),
    tf.keras.layers.Dense(9, activation='relu'),
    #tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(9, activation='relu'),
    #tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

#Compile the model so it can be trained
model.compile(
     loss='sparse_categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)

In [ ]:
h = model.fit(
     train_tf, y_train_id, #class_weight=class_weights,
     epochs=3,
     callbacks=[tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5)]
)


In [ ]:
#Generate predictions

y_pred = np.argmax(model.predict(test_tf), axis=1)

cm = confusion_matrix(y_test_id, y_pred, normalize = 'true')

print(cm)

In [ ]:
print(classification_report(y_test_id, y_pred))

In [ ]:
y_pred_labels = ids_to_names(y_pred)
y_test_labels = ids_to_names(y_test_id)

In [ ]:
y_unique = ["Negative", "Neutral", "Positive"]
cm = confusion_matrix(y_test_labels, y_pred_labels, labels = y_unique, normalize='true')

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=y_unique)
disp.plot(xticks_rotation='vertical',)

In [ ]:
# from joblib import dump
# dump(model, 'spam_classifier.joblib')